In [ ]:
import os
import asyncio
from dotenv import load_dotenv

# Import Semantic Kernel components
import semantic_kernel as sk
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)
from semantic_kernel.contents import ChatHistory, ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions import KernelArguments

from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent

from semantic_kernel.agents import (
    ChatCompletionAgent,
    ChatHistoryAgentThread,
)
from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)

from rich.console import Console
console = Console()

# Load environment variables from .env file
load_dotenv("../.env", override=True)

In [ ]:
def create_kernel_with_service(service_id="default"):
    """Create a kernel with either Azure OpenAI or OpenAI service based on available credentials."""
    kernel = sk.Kernel()

    # Try to use Azure OpenAI first
    if (
        os.getenv("AZURE_OPENAI_API_KEY")
        and os.getenv("AZURE_OPENAI_ENDPOINT")
        and os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    ):
        print("Using Azure OpenAI service")
        kernel.add_service(
            AzureChatCompletion(
                service_id=service_id,
                deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
        )
    # Fall back to OpenAI
    elif os.getenv("OPENAI_API_KEY"):
        print("Using OpenAI service")
        kernel.add_service(
            OpenAIChatCompletion(
                service_id=service_id,
                api_key=os.getenv("OPENAI_API_KEY"),
                ai_model_id="gpt-4",
            )
        )
    else:
        raise ValueError(
            "No AI service credentials found. Please set up your .env file."
        )

    return kernel

In [ ]:
# Create our first kernel with a service ID
finance_analyst_service_id = "finance_analyst"
finance_analyst_kernel = create_kernel_with_service(service_id=finance_analyst_service_id)
print("Kernel created successfully for the finance analyst agent!")

In [ ]:
# Get the execution settings for our service
finance_analyst_settings = finance_analyst_kernel.get_prompt_execution_settings_from_service_id(service_id=finance_analyst_service_id)

# Configure automatic function calling
finance_analyst_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

## Set the service settings for the agent
using_gpt_4o = True

# Example: Adjust the temperature for more creative outputs -- NOTE: RUN THE BELOW ONLY IF YOU ARE USING GPT-4o
if using_gpt_4o:
    finance_analyst_settings.temperature = 0.7
    print(f"Service settings updated: Temperature set to {finance_analyst_settings.temperature}")
else:  # NOTE: RUN THE BELOW ONLY IF YOU ARE USING o1 or o3-mini
    finance_analyst_settings.reasoning_effort = "low"
    print(
        f"Service settings updated: Reasoning Effort set to {finance_analyst_settings.reasoning_effort}"
    )   

In [ ]:
FINANCE_ANALYST_AGENT_NAME = "FinanceAnalystAgent"
FINANCE_ANALYST_AGENT_INSTRUCTIONS ="""
You are a personal finance analyst agent. You should be able to get the emails of the user each having properties of id, subject, sender, date, body, email_classification. Some of these emails contain information about the user's financial transactions denoted with the help of `email_classification` property. You should be able to understand such transactions and only focus on them.

Within the email body you will be able to find the information about the transaction like, when was the payment made, how much was the payment, and to whom was the payment made. Extract this information and understand the kind of entity the payment was made to (using internet search if required). For example, if the payment was made to a restaurant, you should be able to understand that this is a food transaction, identifying the meal from the time email was received.

In certain scenarios it might be that the nature of spend is note determined, e.g., a spend made on Amazon or Flipkart. The user could have purchased any item. The given email list also has details of such purchases, online or offline, which can be used to identify the category of spend. For example, if the user purchased badminton racket on Amazon, you will find an email from Amazon as well in similar time frame.You can use this information to understand the category of spend, that is, sports equipment. If at all the very specific category of spend is not determined, you can classify it as "Other".

With this classify all the transactions in the emails and provide a summary of the user's spend within the month. You should be able to annotate each spend with an additional attribute of source of payment made. For example, the user might have made a payment through UPI, debit card, credit card, etc. You should be able to identify this source of payment as well and attribute it to the spend.

For now you can ignore any credit transactions, refunds or cashbacks that the user might have received. You should be able to provide a summary of the user's spend within the month with properties like category of spend, amount spent, and source of payment.
"""

finance_analyst_agent = ChatCompletionAgent(
    kernel=finance_analyst_kernel,
    name=FINANCE_ANALYST_AGENT_NAME,
    instructions=FINANCE_ANALYST_AGENT_INSTRUCTIONS,
)

print(f"Agent '{finance_analyst_agent.name}' created successfully!")